<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/audio_clasifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install tensorflow librosa

# Importar bibliotecas necesarias
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Descargar y preparar el conjunto de datos
# Para este ejemplo, asumimos que tienes un conjunto de datos con archivos de audio de ladridos y maullidos
# Organiza tus datos en carpetas 'perro' y 'gato' dentro de una carpeta 'datos'
# Sube la carpeta 'datos' a tu Google Drive y móntalo en Colab

from google.colab import drive
drive.mount('/content/drive')

# Ruta a la carpeta de datos
data_path = '/content/drive/My Drive/datos/'

# Función para extraer características de los archivos de audio
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, resample=True)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Error al procesar {file_path}: {e}")
        return None
    return mfccs_scaled

# Preparar datos y etiquetas
features = []
labels = []

for label in ['perro', 'gato']:
    folder_path = os.path.join(data_path, label)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        data = extract_features(file_path)
        if data is not None:
            features.append(data)
            labels.append(label)

# Convertir a arrays de numpy
X = np.array(features)
y = np.array(labels)

# Codificar etiquetas
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Construir el modelo
model = Sequential()
model.add(Dense(256, input_shape=(40,), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc:.2f}')